Importing libraries

In [1]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression as LR
from sklearn.feature_extraction.text import TfidfVectorizer as TV
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier as GB
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier as KNC
from Database import Database
from numpy import array
from sklearn.naive_bayes import MultinomialNB as NB
import sqlite3
import tqdm
import random

And the data.

If the currency would increases on the next period after the news is launched -> labeled + 1
If the currnecy would decreases on the next period after the news is launched -> labeled - 1

In [2]:
conn = sqlite3.connect('berita.db')
c = conn.cursor()
c.execute("SELECT Text,Sentiment,Date,Clock From Berita ")
result = c.fetchall()

In [3]:
X = []
y = []
for I in result:
    X.append(I[0])
    y.append(I[1])

A good evaluation metric should be directly motivated by the problem.

We find three charactherization:
    
    (1) Both false positive and false negative are harmful
    
    (2) The number of positive point and negative point fluctuates as condition of economy
    
    (3) The number of positive point and negative is not equal and the evaluation metric accomodates them.
    
We would like to think that mrc is the most suitable parameter compared to other parameters like F1 but also provides more intuitive information than AUROC.

In [4]:
def mrc(pred,Y):
    
    pred = array(pred)
    Y    = array(Y)
    
    TP, FP , TN, FN = 0,0,0,0
    
    for I in range(len(pred)):
        if pred[I] == Y[I]:
            if pred[I] == 1:
                TP += 1
            else:
                TN += 1
        else:
            if pred[I] == -1:
                FN += 1
            else:
                FP += 1
    print(TP,FP,TN,FN)
    try:
        return ((TP*TN)-(FP*FN)) / ((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))**(0.5)
    except:
        return 0

In [5]:
#Custom class for GradientBoosting
class Boosting():
    def __init__(self):
        self.clf = GB()
    def fit(self,X,y):
        self.clf.fit(X,y)
    def predict(self,X):
        m = int(X.shape[0] ** (0.5))
        pred = []
        for I in range(m):
            pred.extend(self.clf.predict(X[I*X.shape[0]//m:(I+1)*X.shape[0]//m].toarray()))
        return pred
#Custom class for K Nearest Neighbor
class Neighbors:
    def __init__(self):
        self.clf = KNC()
    def fit(self,X,y):
        self.clf.fit(X,y)
    def predict(self,X):
        m = int(X.shape[0] ** (0.5))
        pred = []
        for I in range(m):
            pred.extend(self.clf.predict(X[I*X.shape[0]//m:(I+1)*X.shape[0]//m]))
        return pred

In [6]:
clfOption = [Boosting(),LR(n_jobs = -1),NB(),LinearSVC(),Neighbors(),RFC()]

We empoy nested cross validation.

The first cross validation is to find the best feature selection

The second cross validation is to evaluate the model.

Both are 5-fold cross validation.

However, on the training set of each cross validation, we would under sample the majority class to ensure 50/50 balance of the training set.

In [7]:
def splitGroup(kFold):
    #Me too lazy to plug in sklearn
    assert len(X) == len(y)
    index = [int(I) for I in range(len(X))]
    random.shuffle(index)
    group = [index[len(index)*I//kFold:len(index)*(I+1)//kFold] for I in range(kFold)]
    return group

In [8]:
def balancedTrain(X,y):
    balancedX = []
    balancedY = []
    
    index = {}
    
    assert len(X) == len(y)
    
    for I in range(len(X)):
        if y[I] not in index:
            index[y[I]] = []
        index[y[I]].append(I)
    minimumPoint = min([len(I) for I in index.values()])    
    
    for I in index:
        chosen = random.sample(index[I],k = minimumPoint)
        for J in chosen:
            balancedX.append(X[J])
            balancedY.append(y[J])
    return balancedX, balancedY

In [9]:
#First cross validation
group = splitGroup(5)
XkFold = [[X[J] for J in K]for K in group]
YkFold = [[y[J] for J in K]for K in group]
counterList = []
selectList = []
mreTotal = []
t = tqdm.tqdm(total=5)
for I in range(5):
    xTrain = []
    yTrain = []
    
    xTest  = []
    yTest = []
    
    for J in range(5):
        if J == I:
            for L in XkFold[J]:
                sentence = L
                newSentence = ""
                for K in sentence:
                    if K.isalpha():
                        newSentence += K
                    else:
                        newSentence += " "
                xTest.append(newSentence)
            yTest.extend(YkFold[J])
        else:
            for L in XkFold[J]:
                sentence = L
                newSentence = ""
                for K in sentence:
                    if K.isalpha():
                        newSentence += K
                    else:
                        newSentence += " "
                xTrain.append(newSentence)
            yTrain.extend(YkFold[J])
            
    xTrain , yTrain = balancedTrain(xTrain,yTrain)
    counterList.append(TV(ngram_range=(2,2),min_df=5)) 
    trainVector = counterList[-1].fit_transform(xTrain)
    testVector  = counterList[-1].transform(xTest)
    
    selectList.append(SelectKBest(chi2, k = 10000))
    
    trainVector = selectList[-1].fit_transform(trainVector,yTrain)
    testVector  = selectList[-1].transform(testVector)
            
    
    mreTotal.append(0)
    for J in clfOption:
        J.fit(trainVector,yTrain)
        prediction = J.predict(testVector)
        mreTotal[-1] += mrc(prediction,yTest)
    t.update()

  0%|                                                                                           | 0/5 [00:00<?, ?it/s]

6436 4582 2773 2412
5148 3216 4139 3700
5000 2966 4389 3848
5207 3081 4274 3641
4686 3124 4231 4162
4226 2518 4837 4622


 20%|████████████████▍                                                                 | 1/5 [04:53<19:32, 293.18s/it]

6143 4332 2926 2802
5136 3237 4021 3809
4882 2853 4405 4063
5228 3089 4169 3717
5075 3339 3919 3870
4273 2486 4772 4672


 40%|████████████████████████████████▊                                                 | 2/5 [09:39<14:33, 291.13s/it]

...

 60%|█████████████████████████████████████████████████▏                                | 3/5 [14:35<09:45, 292.68s/it]

6511 4642 2685 2365
5153 3137 4190 3723
4919 2833 4494 3957
5258 3007 4320 3618
5042 3448 3879 3834
4616 2718 4609 4260


 80%|█████████████████████████████████████████████████████████████████▌                | 4/5 [19:54<05:00, 300.46s/it]

6672 4905 2444 2183
5175 3212 4137 3680
4967 2913 4436 3888
5271 3092 4257 3584
4865 3260 4089 3990
4412 2664 4685 4443


100%|██████████████████████████████████████████████████████████████████████████████████| 5/5 [25:17<00:00, 307.38s/it]

In [10]:
print("Average mre are")
print([I/5 for I in mreTotal])
print("The best feature selection is number %d"%(mreTotal.index(max(mreTotal))))

Average mre are
[0.1653365967017511, 0.15485612882219488, 0.17406455666515908, 0.17070805113352958, 0.16419834927037513]
The best feature selection is number 2


In [11]:
index = mreTotal.index(max(mreTotal))

In [12]:
#Second cross validation to evaluate result
group = splitGroup(5)
XkFold = [[X[J] for J in K]for K in group]
YkFold = [[y[J] for J in K]for K in group]
mreTotal = [0 for I in clfOption]
t = tqdm.tqdm(total=5)
for I in range(5):
    xTrain = []
    yTrain = []
    
    xTest  = []
    yTest = []
    
    for J in range(5):
        if J == I:
            for L in XkFold[J]:
                sentence = L
                newSentence = ""
                for K in sentence:
                    if K.isalpha():
                        newSentence += K
                    else:
                        newSentence += " "
                xTest.append(newSentence)
            yTest.extend(YkFold[J])
        else:
            for L in XkFold[J]:
                sentence = L
                newSentence = ""
                for K in sentence:
                    if K.isalpha():
                        newSentence += K
                    else:
                        newSentence += " "
                xTrain.append(newSentence)
            yTrain.extend(YkFold[J])
            
    xTrain , yTrain = balancedTrain(xTrain,yTrain)
    trainVector = counterList[index].transform(xTrain)
    testVector  = counterList[index].transform(xTest)
        
    trainVector = selectList[index].transform(trainVector)
    testVector  = selectList[index].transform(testVector)

    for J in range(len(clfOption)):
        clfOption[J].fit(trainVector,yTrain)
        prediction = clfOption[J].predict(testVector)
        mreTotal[J] += mrc(prediction,yTest)
    t.update()


  0%|                                                                                           | 0/5 [00:00<?, ?it/s]


6338 4621 2783 2461
5670 2705 4699 3129
5818 2063 5341 2981
6028 2263 5141 2771
5309 3171 4233 3490
4540 2270 5134 4259


 20%|████████████████▍                                                                 | 1/5 [04:40<18:42, 280.56s/it]

6301 4600 2759 2543
5863 2759 4600 2981
5923 2109 5250 2921
6201 2268 5091 2643
5223 3131 4228 3621
4618 2315 5044 4226


 40%|████████████████████████████████▊                                                 | 2/5 [09:24<14:04, 281.50s/it]

...

 60%|█████████████████████████████████████████████████▏                                | 3/5 [13:52<09:15, 277.53s/it]

6627 4795 2494 2287
5843 2714 4575 3071
5929 2134 5155 2985
6160 2307 4982 2754
5223 3018 4271 3691
4796 2381 4908 4118


 80%|█████████████████████████████████████████████████████████████████▌                | 4/5 [18:31<04:38, 278.08s/it]

6509 4511 2687 2497
5913 2749 4449 3093
6014 2066 5132 2992
6229 2279 4919 2777
5206 2973 4225 3800
4766 2441 4757 4240


100%|██████████████████████████████████████████████████████████████████████████████████| 5/5 [23:10<00:00, 278.09s/it]

In [13]:
name = ["Gradient Boosting","Logistic Regression","Naive Bayes","Linear SVC","K nearest neighbor","Random forest"]
assert len(name) == len(mreTotal)
for I in range(len(name)):
    print("%s -> %f" % (name[I],mreTotal[I]/5))

Gradient Boosting -> 0.100321
Logistic Regression -> 0.278953
Naive Bayes -> 0.378425
Linear SVC -> 0.378479
K nearest neighbor -> 0.169274
Random forest -> 0.203007


Chronological prediction

In [14]:
d = Database()
data = {}
label = {}
for I in result:
    session = d.cariSesi(I[2],I[3])[0]
    if session not in data:
        data[session] = []
        label[session] = I[1]
    data[session].append(I[0])


chronology = list(data.keys())

In [15]:
len(chronology)

492

In [16]:
def splitChronologyGroup(kFold):
    #Me too lazy to plug in sklearn
    index = [int(I) for I in range(len(chronology))]
    random.shuffle(index)
    group = [index[len(index)*I//kFold:len(index)*(I+1)//kFold] for I in range(kFold)]
    return group

In [17]:
#Second cross validation to evaluate result
group = splitChronologyGroup(5)
XkFold = [[chronology[J] for J in K]for K in group]
YkFold = [[label[chronology[J]] for J in K]for K in group]

In [19]:
mreTotal = [0 for I in clfOption]
#Second cross validation to evaluate result
group = splitChronologyGroup(5)
XkFold = [[chronology[J] for J in K]for K in group]
YkFold = [[label[chronology[J]] for J in K]for K in group]

t = tqdm.tqdm(total=5)
for I in range(5):
    xTrain = []
    yTrain = []
    
    xTest  = []
    yTest = []
    
    for J in range(5):
        if J != I:
            for K in XkFold[J]:
                for L in data[K]:
                    sentence = L
                    newSentence = ""
                    for M in sentence:
                        if M.isalpha():
                            newSentence += M
                        else:
                            newSentence += " "
                    xTrain.append(newSentence)
                    yTrain.append(label[K])
        else:
            for K in XkFold[J]:
                xTest.append([])
                for L in data[K]:
                    sentence = L
                    newSentence = ""
                    for M in sentence:
                        if M.isalpha():
                            newSentence += M
                        else:
                            newSentence += " "
                    xTest[-1].append(newSentence)
                yTest.append(label[K])
            
    xTrain , yTrain = balancedTrain(xTrain,yTrain)
    trainVector = counterList[index].transform(xTrain)
    testVector  = [counterList[index].transform(J) for J in xTest]
        
    trainVector = selectList[index].transform(trainVector)
    testVector  = [selectList[index].transform(J) for J in testVector]

    for J in range(len(clfOption)):
        clfOption[J].fit(trainVector,yTrain)
        prediction = [[-1,1][sum(clfOption[J].predict(K))>0] for K in testVector]
        mreTotal[J] += mrc(prediction,yTest)
    t.update()


  0%|                                                                                           | 0/5 [00:00<?, ?it/s]


46 31 14 7
40 9 36 13
45 0 45 8
49 2 43 4
48 13 32 5
16 4 41 37


 20%|████████████████▍                                                                 | 1/5 [06:30<26:00, 390.01s/it]

58 35 3 2
46 4 34 14
55 1 37 5
53 1 37 7
48 9 29 12
21 0 38 39


 40%|████████████████████████████████▊                                                 | 2/5 [12:23<18:56, 378.99s/it]

...

 60%|█████████████████████████████████████████████████▏                                | 3/5 [17:58<12:11, 365.99s/it]

47 32 14 5
43 13 33 9
44 1 45 8
46 4 42 6
41 7 39 11
11 0 46 41


 80%|█████████████████████████████████████████████████████████████████▌                | 4/5 [23:20<05:52, 352.80s/it]

43 39 11 6
37 11 39 12
43 3 47 6
47 3 47 2
41 17 33 8
19 0 50 30


100%|██████████████████████████████████████████████████████████████████████████████████| 5/5 [28:09<00:00, 333.62s/it]

In [20]:
name = ["Gradient Boosting","Logistic Regression","Naive Bayes","Linear SVC","K nearest neighbor","Random forest"]
assert len(name) == len(mreTotal)
for I in range(len(name)):
    print("%s -> %f" % (name[I],mreTotal[I]/5))

Gradient Boosting -> 0.132201
Logistic Regression -> 0.542771
Naive Bayes -> 0.830636
Linear SVC -> 0.821913
K nearest neighbor -> 0.587157
Random forest -> 0.330527
